In [1]:
#yeeeeedsffdfddsfad
import numpy as np
np.random.seed(2020)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss
from imageio import imread
#numpy.array(Image.fromarray(arr).resize()) for image resizing

#image resolution

#globals for rows and cols since we will always be doing images of the same size
rows = 80
cols = 60


Using TensorFlow backend.


In [10]:
def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')
        
def read_and_normalize_train_data():
    cache_path = os.path.join('cache', 'train_r_' + str(80) + '_c_' + str(60) + '_t_' + str(3) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target, driver_id, unique_drivers = load_train()
        cache_data((train_data, train_target, driver_id, unique_drivers), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target, driver_id, unique_drivers) = restore_data(cache_path)

    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    train_data = train_data.reshape(train_data.shape[0], 3, 80, 60)
    train_target = np_utils.to_categorical(train_target, 10)
    train_data = train_data.astype('float32')
    train_data /= 255
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, driver_id, unique_drivers


def read_and_normalize_test_data():
    cache_path = os.path.join('cache', 'test_r_' + str(80) + '_c_' + str(60) + '_t_' + str(3) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_id = load_test()
        cache_data((test_data, test_id), cache_path)
    else:
        print('Restore test from cache!')
        (test_data, test_id) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(test_data.shape[0], 3, 80, 60)
    test_data = test_data.astype('float32')
    test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id

def get_driver_data():
    dr = dict()
    path = ('driver_imgs_list.csv')
    #print(path)
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr

def load_train():
    X_train = []
    y_train = []
    driver_id = []
    start_time = time.time()
    driver_data = get_driver_data()

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('train', 'c' + str(j), '*.jpg')
        #print(path)
        #return
        files = glob.glob(path)
        for fl in files:
            img = cv2.imread(fl)
            #img = get_im_cv2_mod(fl, img_rows, img_cols, color_type)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[os.path.basename(fl)])

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    #print(unique_drivers)
    
    return X_train, y_train, driver_id, unique_drivers

def load_test():
    print('Read test images')
    start_time = time.time()
    path = os.path.join('test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        img = cv2.imread(fl)
        X_test.append(img)
        X_test_id.append(os.path.basename(fl))
        total += 1
        if total%thr == 0:
            print('Read {} images from {}'.format(total, len(files)))
    
    print('Read test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_test, X_test_id

#testing

def copy_selected_drivers(train_data, train_target, driver_id, driver_list):
    data = []
    target = []
    index = []
    for i in range(len(driver_id)):
        if driver_id[i] in driver_list:
            data.append(train_data[i])
            target.append(train_target[i])
            index.append(i)
    data = np.array(data, dtype=np.float32)
    #print('hi2',data.shape)
    target = np.array(target, dtype=np.float32)
    index = np.array(index, dtype=np.uint32)
    return data, target, index

def create_model():
    model = Sequential()
    #model.add(Convolution2D(32, 3, 3, border_mode='same', init='he_normal', input_shape=(1, 80, 60,)))
    #input_shape=(1, 60, 80)
    model.add(Convolution2D(32, (3, 3), input_shape=(3, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(8, 8), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
train_data, train_target, driver_id, unique_drivers = read_and_normalize_train_data()
test_data, test_id = read_and_normalize_test_data()

Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Read train data time: 9.85 seconds
Unique drivers: 26
Directory doesnt exists
Train shape: (22424, 3, 80, 60)
22424 train samples
Read test images
Read 7972 images from 79726
Read 15944 images from 79726
Read 23916 images from 79726
Read 31888 images from 79726
Read 39860 images from 79726
Read 47832 images from 79726
Read 55804 images from 79726
Read 63776 images from 79726
Read 71748 images from 79726
Read 79720 images from 79726
Read test data time: 32.57 seconds
Directory doesnt exists
Test shape: (79726, 3, 80, 60)
79726 test samples


In [15]:
#d = get_driver_data()
#x,y,z,c = load_train()
#x,y = load_test()
#cv2.imshow('img',x[0])
#print(y)
#print(z)
#print(c)

batch_size = 32
epochs = 1

#X_train = X_train.reshape(X_train.shape[0], 3, 80, 60)
#Y_train = np_utils.to_categorical(Y_train, 10)


yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)



model = create_model()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

Start Single Run
Split train:  21601 21601
Split valid:  823 823
Train drivers:  ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
Test drivers:  ['p081']
(21601, 3, 80, 60) (21601, 10)
(823, 3, 80, 60) (823, 10)
Train on 21601 samples, validate on 823 samples
Epoch 1/1
823/823 [==============================] - 0s 156us/step
Score log_loss:  2.289948450113994
79726/79726 [==============================] - 10s 126us/step
Final log_loss: 2.289948450113994, rows: 80 cols: 60 epochs: 1
